In [37]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import math
from urllib.parse import quote
import pandas as pd

In [38]:
def getPage(url):
#     url = "https://en.wikipedia.org/wiki/New_Delhi"
    page = urlopen(url)
    page = page.read().decode("utf-8")
    return page

In [39]:
def preProcess(page):
    soup = BeautifulSoup(page, 'html.parser')
    clean_soup = soup.get_text()
    
    whitespaces = re.compile("\n[\n]+\n")
    refs = re.compile("\[.*?\]")
    page = re.sub(whitespaces, "\n", clean_soup)
    page = re.sub(refs, " ", page)
    
    paras = ["".join(list(s.strings)).strip()+"\n" for s in soup.find_all('p')]
    paras = [s for s in paras if s != "\n"] # paras
    cleaned_text = [s for s in paras if s!="\n"]
    cleaned_text = "\n".join(cleaned_text)
    return cleaned_text

In [40]:
# clean_soup

In [41]:
sports = []
sports_topics = ['Olympic_Games','Paralympic_Games','Sport_in_India','Euro']

# for topic in sports_topics:
#     cleaned_text = preProcess(getPage("https://en.wikipedia.org/wiki/"+topic))
#     sports.append(cleaned_text)

# for s in sports:
#     print(s)

In [42]:
edu = []
edu_topics = ['The_Language_of_Goldfish','International_Conference_on_Software_Engineering','Education_of_Generation_Z','Prison_education']

# for topic in edu_topics:
#     cleaned_text = preProcess(getPage("https://en.wikipedia.org/wiki/"+topic))
#     edu.append(cleaned_text)
# 
# for e in edu:
#     print(e)

In [43]:
docs = []
data = {
    "docs":[],
    "category":[] 
}
for topic in sports_topics:
    cleaned_text = preProcess(getPage("https://en.wikipedia.org/wiki/"+topic))
    docs.append(cleaned_text)
    data["docs"].append(cleaned_text)
    data["category"].append("sports")
for topic in edu_topics:
    cleaned_text = preProcess(getPage("https://en.wikipedia.org/wiki/"+topic))
    docs.append(cleaned_text)
    data["docs"].append(cleaned_text)
    data["category"].append("education")

# docs

In [44]:
def get_all_unique_words(page1):
    unique_words = set()
    for page in page1:
        words = [w for w in word_tokenize(page) if w.isalnum()] # word tokenize
        unique_words.update(words)
    return unique_words

In [45]:
all_unique_words = get_all_unique_words(docs)

In [46]:
def tokenize(text):
    words = [w.lower() for w in text.split()]
    return words

In [85]:
# vocab = set()

# for text in docs:
#     vocab.update(tokenize(text))
vocab = set(all_unique_words)

unigram_matrix = []

# print("here1")
uni_mat= []
for  text in docs:
    tokenized_text = tokenize(text)
    unigram_count = [tokenized_text.count(unigram) for unigram in vocab]
    unigram_matrix.append(unigram_count)
    
    counts = {w: 0 for w in all_unique_words}
    for i,w  in enumerate(vocab):
        counts[w] = unigram_count[i]
    uni_mat.append(counts)

# print("here2")

# print("Vocab:", list(vocab))
# for i, txt in enumerate(docs):
#     print("Unigram Count: ", unigram_matrix[i])
uni_df = pd.DataFrame(uni_mat)

uni_df.head()

,Future,Netherlands,casuals,344,visor,partnerships,extreme,ta,Channel,date,...,shops,schoolchildren,labour,proper,retail,popular,448,1996,impacts,2019
0,0,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,2,1,1,1,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,2,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,43,0,3,0,3
3,0,0,0,1,0,0,0,0,0,2,...,1,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
def tokenize_bi(text):
    tokens = text.split()
    return [(tokens[i],tokens[i+1]) for i in range(len(tokens)-1)]

In [106]:
# Bigram probability:

bigram_vocab = set()
for text in docs:
    bigram_vocab.update(tokenize_bi(text))

bigram_matrix = []

big_mat = []

for text in docs:
    tokenize_bis = tokenize_bi(text)
    bigram_count = [tokenize_bis.count(bigram) for bigram in bigram_vocab]
    bigram_matrix.append(bigram_count)
    
    
    bg = { (w1,w2):0 for w1,w2 in bigram_vocab}
    for x,y in tokenize_bis:
        bg[(x,y)] = bg[(x,y)]+1
    big_mat.append(bg)
big_mat
# print("Vocab: ", list(bigram_vocab))
# for i, txt in enumerate(docs):
#     print(f"\nDocument {i+1} - {txt}\n")
#     print("Bigram Count: ", bigram_matrix[i])
    
bi_df = pd.DataFrame(big_mat)
bi_df.head()

,"(to, attain)","(Premier, League)","(the, 49)","(the, qualification.[84])","(Games., This)","(or, ""black)","(her, athletic)","(venue, where)","(aimed, to)","(competitive, against)",...,"(in, 23)","(which, was)","(these, games)","(television,, though)","(for, Sochi)","(division, on)","(operating, currency)","(games., They)","(which, still)","(who, became)"
0,1,0,0,0,4,0,0,1,0,1,...,1,8,2,1,1,0,0,0,0,1
1,0,0,0,0,2,1,1,0,0,0,...,0,3,1,0,0,0,0,0,1,0
2,0,4,0,0,0,0,0,0,1,0,...,0,7,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
# idf = log((1+n)/(1+df(t))) + 1 ; where n is no of docs, df(t) is no of docs containing term t

# TF_vocab = set()
# for text in docs:
#     TF_vocab.update(tokenize(text))

TF_vocab = set(all_unique_words)

TF_fre = []


for text in docs:
    tokenized_text = tokenize(text)
    Tf_fre = {tf: tokenized_text.count(tf) for tf in TF_vocab}
    TF_fre.append(Tf_fre)

doc_fre = {tf: sum(1 for doc in TF_fre if doc[tf]>0) for tf in TF_vocab}
inv = {tf: math.log(1+len(docs)/ (1 + doc_fre[tf])) for tf in TF_vocab}


    

TF_matrix = []
for fre in TF_fre:
    xx = {term: tf* inv[term] for term, tf in fre.items()}
    TF_matrix.append(xx)
    


# print("Vocab: ",list(TF_vocab))
# for i, txt in enumerate(docs):
#     print(f"\nDocument {i+1} - {txt}\n")
#     print("TF_IDF Count: ", TF_matrix[i])
    
# idf = {}
# for term, tf in enumerate(fre.items():
#     idf[term]= inv[i]


tf_df = pd.DataFrame(TF_matrix)
tf_df.head()

,Future,races,Netherlands,casuals,344,Lahore,visor,Richmond,partnerships,extreme,...,popular,belief,448,Boxing,1996,which,competitiveness,impacts,2019,the
0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,1.911023,1.609438,1.609438,0.0,0.847298,29.805329,0.000000,1.609438,0.000000,772.726352
1,0.0,2.598566,0.0,0.000000,0.000000,0.0,1.609438,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,1.694596,15.942385,0.000000,0.000000,0.000000,324.354271
2,0.0,6.496415,0.0,1.609438,0.000000,0.0,0.000000,0.0,0.0,0.0,...,41.086992,0.000000,0.000000,0.0,2.541894,29.805329,0.000000,0.000000,3.295837,720.575273
3,0.0,0.000000,0.0,0.000000,1.609438,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,11.783502,1.299283,0.000000,0.000000,349.157833
4,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.693147,0.000000,0.000000,0.000000,10.811809


In [71]:
df = pd.DataFrame(data)
# df

In [100]:
from sklearn.naive_bayes import MultinomialNB

y = df["category"].apply(lambda x: 0 if x =="sports" else 1)
naive_bayes = MultinomialNB()


In [101]:
new_doc = "Cricket is the most popular sports in india"

In [102]:
from collections import Counter
from copy import deepcopy

uni_vector = {w: 0 for w in all_unique_words}
for w, ct in Counter(word_tokenize(new_doc)).items():
    if w in uni_vector: uni_vector[w] = ct

bi_vector = {bg: 0 for bg in bigram_vocab}
bigram_counts = Counter(nltk.bigrams(word_tokenize(new_doc)))
for bigram, count in bigram_counts.items():
    prev_word = bigram[0]
    
    if bigram in bi_vector and prev_word in uni_vector:
        prev_word_count = uni_vector[prev_word]
        bigram_prob = count/prev_word_count
        bi_vector[bigram] = bigram_prob
        
term_freqs = deepcopy(uni_vector)
for term, freq in term_freqs.items():
    term_freqs[term] = freq * inv[term]

print(uni_vector)
print(bi_vector)
print(term_freqs)

{'Future': 0, 'Netherlands': 0, 'casuals': 0, '344': 0, 'visor': 0, 'partnerships': 0, 'extreme': 0, 'ta': 0, 'Channel': 0, 'date': 0, 'Rajasthan': 0, 'neglected': 0, 'innovative': 0, 'Chetan': 0, 'Many': 0, 'woman': 0, 'His': 0, 'Stan': 0, 'ignorant': 0, 'skiers': 0, 'superpower': 0, 'mention': 0, 'allies': 0, 'agrees': 0, 'soccer': 0, 'analysts': 0, 'encouragement': 0, 'kick': 0, 'skating': 0, 'mutual': 0, 'sentence': 0, 'strictly': 0, 'resistance': 0, 'Emile': 0, 'turf': 0, 'proceed': 0, 'budding': 0, 'basketball': 0, 'Ibolya': 0, 'Agency': 0, 'length': 0, 'conducts': 0, 'There': 0, 'Justice': 0, 'level': 0, 'linguistic': 0, 'bowler': 0, 'Provogue': 0, 'danda': 0, 'protest': 0, 'secure': 0, 'olive': 0, 'termed': 0, 'Dhayakattai': 0, 'Budapest': 0, 'ethics': 0, 'victories': 0, 'caused': 0, 'Romans': 0, 'feels': 0, 'performer': 0, 'evident': 0, 'Konstantinos': 0, 'Order': 0, 'temples': 0, 'medallist': 0, 'default': 0, 'jointly': 0, 'Kenya': 0, 'hoisted': 0, 'Pact': 0, 'eligible': 0, '

In [112]:

y = df["category"].apply(lambda x: 0 if x =="sports" else 1)
y

0    0
1    0
2    0
3    0
4    1
5    1
6    1
7    1
Name: category, dtype: int64

In [113]:
# unigram prediction
uni_df = pd.DataFrame(unigram_matrix)
naive_bayes.fit(uni_df, y)
pred = naive_bayes.predict(pd.DataFrame([uni_vector]))[0];
if pred == 0: print("Sports")
else: print("Education")

Education


In [114]:

y = df["category"].apply(lambda x: 0 if x =="sports" else 1)

In [115]:
# Bigram prediction
# uni_df = pd.DataFrame(unigram_matrix)
naive_bayes.fit(bi_df, y)
pred = naive_bayes.predict(pd.DataFrame([bi_vector]))[0];
if pred == 0: print("Sports")
else: print("Education")

Sports


In [119]:
y = df["category"].apply(lambda x: 0 if x =="sports" else 1)

In [121]:
#tfidf prediction
naive_bayes.fit(tf_df, y)
pred = naive_bayes.predict(pd.DataFrame([term_freqs]))[0];
if pred == 0: print("Sports")
else: print("Education")

Education
